Import libs

In [4]:
import pandas as pd
import numpy as np
import functools as ft
import matplotlib.pyplot as plt

Load the data
Before the second step, do "pip3 install xlrd"

In [6]:
date_parser = ft.partial(pd.to_datetime,format ="%d%b%Y")

MUNICIPLAITY_CODE_2_STR = {
    "0114": "Upplands Väsby",
    "0115": "Vallentuna",
    "0117": "Österåker",
    "0120": "Värmdö",
    "0123": "Järfälla",
    "0125": "Ekerö",
    "0126": "Huddinge",
    "0127": "Botkyrka",
    "0128": "Salem",
    "0136": "Haninge",
    "0138": "Tyresö",
    "0139": "Upplands-Bro",
    "0140": "Nykvarn",
    "0160": "Täby",
    "0162": "Danderyd",
    "0163": "Sollentuna",
    "0180": "Stockholm",
    "0181": "Södertälje",
    "0182": "Nacka",
    "0183": "Sundbyberg",
    "0184": "Solna",
    "0186": "Lidingö",
    "0187": "Vaxholm",
    "0188": "Norrtälje",
    "0191": "Sigtuna",
    "0192": "Nynäshamn"
}

DATASET_BASE = "../dataset/"

PERSON = pd.read_csv(DATASET_BASE + "person.csv", index_col="lopnr_new",
                     parse_dates=["birth_date","deathdate"],date_parser=date_parser
                    ).rename(columns={"deathdate":"death_date"})

DIAGNOSES = pd.read_csv(DATASET_BASE + "diagnoses.csv", index_col="lopnr_new",
                       parse_dates=["diagdate","rad_date","kem_date","surg_date"],
                        date_parser=date_parser,dtype={"lkf_dx":np.str}
                       ).replace({"lkf_dx":MUNICIPLAITY_CODE_2_STR})

PSA = pd.read_csv(DATASET_BASE + "psa.csv", index_col="psa_sample_id_new",
                 parse_dates=["psadate"],date_parser=date_parser,dtype={"lkf_psa":np.str}
                 ).replace({"lkf_psa":MUNICIPLAITY_CODE_2_STR})

BIOPPSY = pd.read_csv(DATASET_BASE + "biopsy.csv", index_col="referral_id_new",
                     parse_dates=["bioreferral_date"],date_parser=date_parser,
                     dtype={"lkf_bio":np.str}
                     ).replace({"lkf_bio":MUNICIPLAITY_CODE_2_STR})

CODE_BOOK = pd.read_excel(DATASET_BASE + "Codebook.xlsx",sheetname = ["person","diagnoses","biopsy","psa"])

municipality_code_2_string = {}

Lets take a look what fields are in the different tables

In [186]:
CODE_BOOK["person"]

,Variable,Type,Description,Key
0,lopnr_new,numeric,"Anonymized Unique ID for a person , key for in...",Primary
1,birth_date,date,Date of birth,NaN
2,inc_grp,"numeric, categorical","Income in quartiles, 0-3",NaN
3,death,binary,Information if a man has died. Values: 1 if de...,NaN
4,pc_death,binary,Information if man death cause is registered p...,NaN
5,death_date,date,Date of death,NaN
6,birth_place,"characters, categorical","Birth country, 1=Sweden, 2=Nordic countries, 3...",NaN


In [187]:
CODE_BOOK["diagnoses"]

,Variable,Type,Description,Key
0,lopnr_new,numeric,"Anonymized Unique ID for a person , key for in...",Primary forign(person )
1,diagdate,date,Date of prostate cancer diagnosis,NaN
2,referral_id_new,numeric,Referral id number of the biopsy that is close...,forign(biopsy)
3,trigger_psa_id,numeric,The PSA test that triggers the biopsy. Format...,forign(psa)
4,index_psa_id,numeric,The PSA test that raises concern of cancer. ...,forign(psa)
5,diagnosis_cause,"numeric, categorical",As registered when patients are diagnosed. Va...,NaN
6,gleason_t,"numeric, categorical",The total gleason score found in the biopsy. G...,NaN
7,t_stage,"character, categorical","t-stage of the cancer in biopsy. Values: T0, T...",NaN
8,m_stage,"character, categorical","m-stage of the cancer in biopsy.(Values: M0, M...",NaN
9,n_stage,"character, categorical","n-stage of the cancer in biopsy. Values: N0, N...",NaN


In [188]:
CODE_BOOK["biopsy"]

,Variable,Type,Description,Key
0,lopnr_new,numeric,"Anonymized Unique ID for a person , key for in...",forign(person)
1,cancer_in_biopsy,binary,An indicator if there was diagnosed a cancer i...,NaN
2,bioreferral_date,date,Date of the referral to biopsy,NaN
3,referral_id_new,numeric,"Identification code for a particular biopsy, u...",primary
4,lkf_bio,character,County and municipality where the person lived...,NaN
5,index_psa_id,numeric,The PSA test that raises concern of cancer. ...,forign(psa)
6,trigger_psa_id,numeric,The PSA test that triggers the biopsy. Format...,forign(psa)


In [195]:
CODE_BOOK["psa"].loc[2,"Description"]

'Value of the total PSA level for a particular PSA test'

In [190]:
data_set = PERSON.join(DIAGNOSES)[["birth_date","inc_grp","diagdate","death","pc_death","birth_place"]
                      ].rename(columns={"lkf_dx":"Kommun"})
data_set.to_csv(DATASET_BASE + "experimental_data.csv")

Dataset with person, living place, and first psa visit

In [219]:
first_psa = PSA.set_index(
    ["lopnr_new","psadate"]).sort_index().groupby(level = 0).head(1).rename(
    columns={"lkf_psa":"municipality"}).loc[:,["psa_total","municipality"]]

first_psa = first_psa[first_psa["psa_total"] > 10].join(DIAGNOSES["diagnosis_cause"]).join(
    PERSON[["birth_date","death","pc_death","death_date"]])
first_psa.to_csv(DATASET_BASE + "first_psa_and_death.csv")
#psa_.join(DIAGNOSES).dropna(axis=0,subset=["diagnosis_cause"]).reset_index()

In [8]:
CODE_BOOK["psa"]

,Variable,Type,Description,Key
0,lopnr_new,numeric,"Anonymized Unique ID for a person , key for me...",forign(Person)
1,psadate,date,Date of psa test,NaN
2,psa_total,numeric,Value of the total PSA level for a particular ...,NaN
3,psa_fot,numeric,Free psa level divided by total psa level,NaN
4,lkf_psa,character,County and municipality where the person lived...,NaN
5,psa_sample_id_new,numeric,"Identification code for a particular PSA test,...",primary


In [20]:
test = PERSON.join(DIAGNOSES)

test = test.join(PSA)
test.to_csv("mergetest.csv")